# Zestaw pytań - bazy danych, biblioteka request, pełna aplikacja

Celem tych ćwiczeń, jest utrwalenie wszystkich zebranych do tej pory informacji przy pomocy budowy aplikacji do odzyskiwania informacji na temat liczb. Aplikacja powinna uwzględniać:

* Bazę danych SQLite
* System rejestracji / logowania. Aby uzyskać dostęp do funkcjonalności aplikacji, użytkownik powinien być zalogowany
* Mapowanie obiektów odzyskanych z bazy danych na instancje klas w Pythonie
* Testy jednostkowe

## Opis Numbers API

Numbers API jest serwisem w którym wykonując zapytanie do serwera pod adresem http://numbersapi.com/number/type  możemy podać następujące informację (wstawiając je w miejsce 'number' oraz 'type'):

* `type`: może być podane jako `trivia`, `math`, `date`, lub `year`. Jeśli ten parametr nie jest podany, domyślnie zostanie użyte `trivia`
* `number`: może być podany jako liczba całkowita, słowo `random` lub (jeżeli `type` jest ustawiony jako `date`) data w postaci `month/day` np. `03/29`, `05/23` itd.

## 1). Przygotowanie bazy danych 

Używając modułu `sqlite3`, stwórz bazę danych o nazwie 'numberapp.db' z następującymi tabelami:

In [3]:
import sqlite3
db = sqlite3.connect('numbeapp.db')
sql = db.cursor()

sql.execute("""CREATE TABLE IF NOT EXISTS users (
        username TEXT, 
        password TEXT, 
        requests_count INT
    )""")

db.commit()
sql.execute("""CREATE TABLE IF NOT EXISTS logs (
        username TEXT, 
        date TEXT, 
        type_param TEXT, 
        number_param TEXT
    )""")
db.commit()
db.close()

* Tabela 'users' z kolumnami 'username', 'password', 'requests_count'
* Tabela 'logs' z kolumnami 'username', 'date', 'type_param', 'number_param'

## 2). Przygotowanie odpowiednich modeli

Dokończ implementację następujących klas:

In [14]:
class User:
    '''
    username: nazwa użytkownika,
    password: hasło użytkownika,
    requests_count: ilość requestów wykonanych przez użytkownika
    '''
    def __init__(self, username, password, requests_count):
        self.username = username
        self.password = password
        self.requests_count = requests_count
    


class Log:
    '''
    username: nazwa użytkownika k†óry wykonał zapytanie,
    date: data kiedy zapytanie zostało wykonane
    type_param: co użytkownik wybrał jako `type`,
    number_param: co użytkownik wybrał jako `number` 
    '''
    def __init__(self, username, data, type_param, number_param):
        self.username = username 
        self.data = data
        self.type_param = type_param
        self.number_param = number_param

    def getlo():
        

**Klasy te powinny być wykorzystywane kiedy odzyskujemy informację z bazy danych**.

## 3). Logika aplikacji

Stwórz plik o nazwie `numberapp.py`. Zaimplementuj następujące funkcje:

#### 3.1). `register()`

Przyjmij od użytkownika dane `username` oraz `password`. Dodaj informację na temat nowego użytkownika do bazy danych.

In [8]:
def register():
    
    username = input('Input username: ')
    password = input('Input password: ')
    requests_count = 0
    conn = sqlite3.connect('numbeapp.db')
    cursor = conn.cursor()
    cursor.execute('INSERT INTO users VALUES (?, ?, ?)', (username, password, requests_count))
    conn.commit()
    conn.close()

#### 3.2). `login()`

Przyjmij od użytkownika dane `username` oraz `password`. Zweryfikuj prawidłowość danych. Jeżeli są one poprawnę, zwróć obiekt klasy `User`, w innym wypadku zwróć wartość `False`

In [12]:
def login():
    global username
    username = input('Input username: ')
    password = input('Input password: ')
    conn = sqlite3.connect('numbeapp.db')
    cursor = conn.cursor()
    queryset = cursor.execute('SELECT * from users')
    users = queryset.fetchall()
    conn.close()
    # Converting primitive data structures to composite data types
    user_objects = []
    for user in users:
        user_objects.append(Users(user[0], user[1]))

    # Verifying user's credentials
    for user in user_objects:
        if user.username == username and user.password == password:
            return user
    return False

#### 3.3). `get_number()` 

Przyjmij od użytkownika dane `type` oraz `number`. Przy pomocy modułu `requests` wykonaj odpowiednie zapytanie. Zwróć użytkownikowi tekst otrzymany z serwera. Zapisz informację na temat zapytania do bazy danych do tabeli 'logs'. Zmodyfikuj dane w tabeli 'users' zwiększając o 1 wartość w kolumnie 'requests_count' danego użytkownika.

**Postaraj się napisać metodę w taki sposób, aby przed wykonaniem zapytania do serwera sprawdzić poprawność danych podanych przez użytkownika.** Możesz np. użyć polecenia `assert`, bloku `try...except` lub po prostu instrukcji warunkowych `if`

In [16]:
def get_number():
    import datetime
    try:
        type = input("Napisz Typ(prykład: `trivia`, `math`, `date`, lub `year`): ")
        if type == 'trivia' or type == 'math' or type == 'date' or type == 'year':
            print('type jest zapisano')
        elif not (type == 'trivia' or type == 'math' or type == 'date'):
            type = 'trivia'
            print('the default type is trivia')
        number = input("Możesz napisać słowo 'random' lub liczbę całkowitą, a jeśli wybrałeś' type ' date, to w formie month / day: ")
        
        b = datetime.datetime.now()
        conn = sqlite3.connect(r'requests/numbeapp.db')
        cursor = conn.cursor()
        cursor.execute('INSERT INTO logs VALUES (?, ?, ?, ?)', (username, b, type, number))
        conn.commit()
        cursor.execute(f'UPDATE users set requests_count =+ 1 WHERE username = "{username}" ')
        conn.commit()
        print('Wpis został pomyślnie zaktualizowany')
        cursor.close()

    except sqlite3.Error as error:
        print("Błąd podczas pracy z SQLite", error)
    finally:
        if conn:
            conn.close()
            print("Połączenie z SQLite zamknięte")
    




#### 3.4). `get_logs()`

Odzyskaj z bazy danych z tabeli 'logs' wszystkie informacje. Przekształć je do postaci obiektów klasy `Log`. Zwróć użytkownikowi informację na temat wszystkich logów (format dowolny)

In [15]:
def get_logs():
    conn = sqlite3.connect(r'requests/numbeapp.db')
    cursor = conn.cursor()

    cursor.execute(f"SELECT * FROM logs WHERE username = '{username}'")
    a = cursor.fetchone()
    usernam = a[0]
    date = a[1]
    typ = a[2]
    num = a[3]
    ex = Log(usernam, date, typ, num)
    ex.logo()
    print(ex)
    conn.close()

#### 3.5). `get_users()`

Odzyskaj z bazy danych z tabeli 'users' wszystkie informacje. Przekształć je do postaci obiektów klasy `User`. Zwróć użytkownikowi informację na temat wszystkich użytkowników (format dowolny)

In [17]:
def get_users():
    conn = sqlite3.connect(r'requests/numbeapp.db')
    cursor = conn.cursor()

    cursor.execute(f"SELECT * FROM users WHERE username = '{username}'")
    a = cursor.fetchone()
    usernam = a[0]
    date = a[1]
    req = a[2]
    ex = User(usernam, date, req)
    ex.us()
    print(ex)
    conn.close()
    pass

#### 3.6). Logika

Możesz przetestować swoją aplikację używając poniższego snippetu

In [18]:
user = False
while True:
    if not user:
        print('1. Zarejestruj się')
        print('2. Zaloguj się')
        print('0. Wyjdź z programu')
        option = input('Podaj opcję: ')
        if option == '1':
            register()
        elif option == '2':
            user = login()
        elif option == '0':
            break
    else:
        print('1. Wykonaj zapytanie')
        print('2. Zobacz logi')
        print('3. Zobacz uzytkowników')
        print('0. Wyjdź z programu')
        option = input('Podaj opcję: ')
        if option == '1':
            get_number()
        elif option == '2':
            get_logs()
        elif option == '3':
            get_users()
        elif option == '0':
            break;

1. Zarejestruj się
2. Zaloguj się
0. Wyjdź z programu


## 4). Testy

Stwórz plik `numberapp_tests.py`. Napisz testy jednostkowe do napisanych funkcjonalności.